In [100]:
import pandas as pd
import numpy as np
import requests
import json
from pandas import json_normalize

In [246]:
df = pd.read_csv("final_detail_dataframe.csv").drop(['movie_rank','critic_name', 'critic_org','critic_cn','imdb_name'], axis=1)
df1 = df.groupby('movie_director')['movie_name'].value_counts().reset_index(name='count') \
    .merge(df, how = 'left', on = 'movie_name').drop_duplicates().drop(['movie_director_y'], axis=1)
df1['properties.radius'] = df1['count']
df1["string"] = df1["movie_name"] + ": " + df1["count"].map(str) + np.where(df1["count"]>1, ' votes', ' vote') + ' ' + df1['storyline']
df1_clean = df1.drop(['properties.radius','storyline', 'movie_name', 'count','movie_year','douban_rate','douban_ppl','imdb_rate','imdb_ppl1','rate_dif','ppl_dif'], axis=1)
df1_clean

,movie_director_x,birth_place,geometry.coordinates,string
0,Abbas Kiarostami,"Tehran, Iran","[51.41503, 35.68877]",Certified Copy: 9 votes James Miller has just ...
9,Abbas Kiarostami,"Tehran, Iran","[51.41503, 35.68877]",Ten: 4 votes Forty-two year old famed composer...
13,Abbas Kiarostami,"Tehran, Iran","[51.41503, 35.68877]",Like Someone In Love: 2 votes Like Someone in ...
15,Abdellatif Kechiche,"Tunis, Tunisia","[10.1827, 36.80001]",Blue Is the Warmest Color: 7 votes Adèle is a ...
22,Abderrahmane Sissako,"Kiffa, Mauritania","[-11.39289, 16.75704]",Timbuktu: 9 votes Not far from the ancient Mal...
...,...,...,...,...
1787,Zhang Yimou,"Xi'an, Shaanxi, China","[-94.33519, 30.04732]",Coming Home: 1 vote Sally Bender is the wife o...
1788,Zhang Yimou,"Xi'an, Shaanxi, China","[-94.33519, 30.04732]",House of Flying Daggers: 1 vote During the rei...
1789,Éric Rohmer,"Tulle, Corrèze, France","[1.75657, 45.25911]",The Lady and the Duke: 1 vote An episodic look...
1790,Éric Rohmer,"Tulle, Corrèze, France","[1.75657, 45.25911]",The Romance of Astrea and Celadon: 1 vote In a...


In [239]:
def rate(rate_dif):
    if rate_dif >= 0:
        return '1'
    else:
        return '0'
def ppl(ppl_dif):
    if ppl_dif >= 0:
        return '5'
    else:
        return '3'
def group(num):
    if num == 4:
        return 'Liked by more people in China'
    elif num == 3:
        return 'Liked by less people in America'
    elif num == 2:
        return 'Liked by less people in China'
    else:
        return 'Liked by more people in America'
def color(num):
    if num == 4:
        return '#FA3B00'
    elif num == 3:
        return '#38FFE1'
    elif num == 2:
        return '#FADD5C'
    else:
        return '#0600AD'
def add_text(director): 
    return director + "'s birthplace is " 

In [256]:
df2 = df1.groupby('movie_director_x')['douban_rate','douban_ppl','imdb_rate','imdb_ppl1'].median()
df3 = df1.groupby('movie_director_x')['properties.radius'].sum()
df2['rate_dif']  = df2['douban_rate'] - df2['imdb_rate']
df2['ppl_dif']  = df2['douban_ppl'] - df2['imdb_ppl1']
df2['rate_num'] = df2['rate_dif'].apply(rate)
df2['ppl_num'] = df2['ppl_dif'].apply(ppl)
df2['properties.group_id']  = df2['rate_num'].astype('int') + (df2['ppl_num']).astype('int') - 2
df2['properties.group_name'] = df2['properties.group_id'].apply(group)
df2['properties.color'] = df2['properties.group_id'].apply(color)
df2_clean = df2.drop(['douban_rate', 'douban_ppl','imdb_rate','imdb_ppl1','rate_dif','ppl_dif','rate_num','ppl_num'], axis=1)
output = df2_clean.merge(df3, how='left',on='movie_director_x')
output['properties.radius'] = output['properties.radius'] ** 0.5 * 3
output

/var/folders/9m/w7ffkstj051dnyjnyklwvwmr0000gn/T/ipykernel_44226/1069414224.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df2 = df1.groupby('movie_director_x')['douban_rate','douban_ppl','imdb_rate','imdb_ppl1'].median()


,properties.group_id,properties.group_name,properties.color,properties.radius
movie_director_x,,,,
Abbas Kiarostami,4,Liked by more people in China,#FA3B00,11.618950
Abdellatif Kechiche,4,Liked by more people in China,#FA3B00,7.937254
Abderrahmane Sissako,2,Liked by less people in China,#FADD5C,9.949874
Adam Curtis,2,Liked by less people in China,#FADD5C,3.000000
Adam McKay,1,Liked by more people in America,#0600AD,5.196152
...,...,...,...,...
Yann Arthus-Bertrand,2,Liked by less people in China,#FADD5C,3.000000
Yervant Gianikian and Angela Ricci Lucchi,2,Liked by less people in China,#FADD5C,3.000000
Yorgos Lanthimos,2,Liked by less people in China,#FADD5C,5.196152


In [268]:
output1 = df1_clean.merge(output, how = 'left', on = 'movie_director_x')
output1['properties.headline'] = output1['movie_director_x'].apply(add_text) + output1['birth_place'] + '.'
output1

,movie_director_x,birth_place,geometry.coordinates,string,properties.group_id,properties.group_name,properties.color,properties.radius,properties.headline
0,Abbas Kiarostami,"Tehran, Iran","[51.41503, 35.68877]",Certified Copy: 9 votes James Miller has just ...,4,Liked by more people in China,#FA3B00,11.618950,"Abbas Kiarostami's birthplace is Tehran, Iran."
1,Abbas Kiarostami,"Tehran, Iran","[51.41503, 35.68877]",Ten: 4 votes Forty-two year old famed composer...,4,Liked by more people in China,#FA3B00,11.618950,"Abbas Kiarostami's birthplace is Tehran, Iran."
2,Abbas Kiarostami,"Tehran, Iran","[51.41503, 35.68877]",Like Someone In Love: 2 votes Like Someone in ...,4,Liked by more people in China,#FA3B00,11.618950,"Abbas Kiarostami's birthplace is Tehran, Iran."
3,Abdellatif Kechiche,"Tunis, Tunisia","[10.1827, 36.80001]",Blue Is the Warmest Color: 7 votes Adèle is a ...,4,Liked by more people in China,#FA3B00,7.937254,"Abdellatif Kechiche's birthplace is Tunis, Tun..."
4,Abderrahmane Sissako,"Kiffa, Mauritania","[-11.39289, 16.75704]",Timbuktu: 9 votes Not far from the ancient Mal...,2,Liked by less people in China,#FADD5C,9.949874,"Abderrahmane Sissako's birthplace is Kiffa, Ma..."
...,...,...,...,...,...,...,...,...,...
609,Zhang Yimou,"Xi'an, Shaanxi, China","[-94.33519, 30.04732]",Coming Home: 1 vote Sally Bender is the wife o...,4,Liked by more people in China,#FA3B00,6.000000,"Zhang Yimou's birthplace is Xi'an, Shaanxi, Ch..."
610,Zhang Yimou,"Xi'an, Shaanxi, China","[-94.33519, 30.04732]",House of Flying Daggers: 1 vote During the rei...,4,Liked by more people in China,#FA3B00,6.000000,"Zhang Yimou's birthplace is Xi'an, Shaanxi, Ch..."
611,Éric Rohmer,"Tulle, Corrèze, France","[1.75657, 45.25911]",The Lady and the Duke: 1 vote An episodic look...,2,Liked by less people in China,#FADD5C,5.196152,"Éric Rohmer's birthplace is Tulle, Corrèze, Fr..."
612,Éric Rohmer,"Tulle, Corrèze, France","[1.75657, 45.25911]",The Romance of Astrea and Celadon: 1 vote In a...,2,Liked by less people in China,#FADD5C,5.196152,"Éric Rohmer's birthplace is Tulle, Corrèze, Fr..."


In [270]:
final_output = output1.groupby('crit_cn')['string']\
    .apply(lambda x: "<div id='movie'><h1><b>Selected Movies</b></h1><P>{0}</P></div>".format('</p><p> '.join(x))).reset_index(name='properties.article')
#I don't know why my lambda will break when I try to include the storyline

KeyError: 'crit_cn'

In [245]:
from ast import literal_eval
final_output['geometry.coordinates'] = final_output['geometry.coordinates'].apply(literal_eval)
final_output2 = final_output.drop(['movie_director_x','birth_place'], axis=1)
final_output2.to_csv("final.csv")

/var/folders/9m/w7ffkstj051dnyjnyklwvwmr0000gn/T/ipykernel_44226/1310436416.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['geometry.coordinates'] = final_df['geometry.coordinates'].apply(literal_eval)


In [224]:
def process_to_geojson(file):
    geo_data = {"type": "FeatureCollection", "features":[]}
    for row in file:
        this_dict = {"type": "Feature", "properties":{}, "geometry": {}}
        for key, value in row.items():
            key_names = key.split('.')
            if key_names[0] == 'geometry':
                this_dict['geometry'][key_names[1]] = value
            if str(key_names[0]) == 'properties':
                this_dict['properties'][key_names[1]] = value
        geo_data['features'].append(this_dict)
    return geo_data
ok_json = json.loads(final_output2.to_json(orient='records'))
geo_format = process_to_geojson(ok_json)

In [225]:
ok_json

[{'properties.article': "<div id='movie'><h1><b>Selected Movies</b></h1><P>Certified Copy: 9 votes</p><p>Ten: 4 votes</p><p>Like Someone In Love: 2 votes</P></div>",
  'geometry.coordinates': [51.41503, 35.68877],
  'geometry.type': 'Point',
  'properties.headline': "Abbas Kiarostami's birthplace is Tehran, Iran.",
  'properties.group_id': 4,
  'properties.group_name': 'Liked by more people in China',
  'properties.color': '#FA3B00',
  'properties.radius': 3.8729833462},
 {'properties.article': "<div id='movie'><h1><b>Selected Movies</b></h1><P>Blue Is the Warmest Color: 7 votes</P></div>",
  'geometry.coordinates': [10.1827, 36.80001],
  'geometry.type': 'Point',
  'properties.headline': "Abdellatif Kechiche's birthplace is Tunis, Tunisia.",
  'properties.group_id': 4,
  'properties.group_name': 'Liked by more people in China',
  'properties.color': '#FA3B00',
  'properties.radius': 2.6457513111},
 {'properties.article': "<div id='movie'><h1><b>Selected Movies</b></h1><P>Timbuktu: 9 v

In [226]:
#Variable name
with open('geo-data.js', 'w') as outfile:
    outfile.write("infoData = ")
#geojson output
with open('geo-data.js', 'a') as outfile:
    json.dump(geo_format, outfile)